# Darcy equation: exercise 2

Let $\Omega=(0,1)^2$ with boundary $\partial \Omega$ and outward unit normal ${\nu}$. Given 
$k=I$ the matrix permeability and $g$ a vector source term, we want to solve the following problem: find $({q}, p)$ such that
$$
\left\{
\begin{array}{ll}
\begin{array}{l} 
k^{-1} {q} + \nabla p = {g}\\
\nabla \cdot {q} = 0
\end{array}
&\text{in } \Omega
\end{array}
\right.
$$
with boundary conditions:
$$ p = 0 \text{ on } \partial_{top} \Omega \qquad p = 1 \text{ on } \partial_{bottom} \Omega \qquad \nu \cdot q = 0 \text{ on } \partial_{left} \Omega \cup \partial_{right} \Omega$$

We present *step-by-step* how to create the grid, declare the problem data, and finally solve the problem.

First we import some of the standard modules.

In [1]:
import numpy as np
import scipy.sparse as sps

import porepy as pp
import pygeon as pg

We create now the grid, in this example we consider a 2-dimensional grid.

In [2]:
mesh_size = 0.05 * 2
# creation of the grid
sd = pg.unit_grid(2, mesh_size, as_mdg=False)
# compute the geometrical properties of the grid
sd.compute_geometry()

With the following code we set the data, in particular the permeability tensor and the boundary conditions. Since we need to identify each side of $\partial \Omega$ we need few steps.

In [3]:
key = "flow"

# declare the discretization objects, useful to setup the data
rt0 = pg.RT0(key)
p0 = pg.PwConstants(key)

# set up the data for the problem
data = {}

# unitary permeability tensor
perm = pp.SecondOrderTensor(np.ones(sd.num_cells))
parameters = {
    "second_order_tensor": perm,
}
pp.initialize_data(sd, data, key, parameters)

# with the following steps we identify the portions of the boundary
# to impose the boundary conditions
left = np.isclose(sd.face_centers[0, :], 0)
right = np.isclose(sd.face_centers[0, :], 1)
left_right = np.logical_or(left, right)

top = np.isclose(sd.face_centers[1, :], 0)
bottom = np.isclose(sd.face_centers[1, :], 1)
top_bottom = np.logical_or(top, bottom)

ess_p_dofs = np.zeros(p0.ndof(sd), dtype=bool)


# compute the pressure boundary condition, which is a natural condition for the RT0 space
def p_bc(x):
    return 1 - x[1]


bc_val = -rt0.assemble_nat_bc(sd, p_bc, top_bottom)
bc_ess = np.hstack((left_right, ess_p_dofs))


# compute the source term
def vector_source_fct(x):
    return np.array([0, -2, 0])


mass = rt0.assemble_mass_matrix(sd)
vector_source = mass @ rt0.interpolate(sd, vector_source_fct)

print(bc_val)

[-1. -0. -0.  1. -0. -0. -0. -0. -0.  0. -0. -0. -1. -0. -0. -1. -0. -0.
 -1. -0. -0. -1. -0. -0. -1. -0. -0. -1. -0. -0. -1. -0. -0. -1. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0

Once the data are assigned to the grid, we construct the matrices. In particular, the linear system associated with the equation is given as
$$
\left(
\begin{array}{cc} 
M & -B^\top\\
B & 0
\end{array}
\right)
\left(
\begin{array}{c} 
q\\ 
p
\end{array}
\right)
=\left(
\begin{array}{c} 
p_{\partial} + g\\ 
0
\end{array}
\right)
$$<br>
where $p_{\partial}$ is the vector associated to the pressure boundary contions. Once the matrix is created, we also construct the right-hand side containing the boundary conditions.

In [4]:
# construct the local matrices
mass_rt0 = rt0.assemble_mass_matrix(sd, data)
mass_p0 = p0.assemble_mass_matrix(sd, data)
div = mass_p0 @ rt0.assemble_diff_matrix(sd)

# assemble the saddle point problem
spp = sps.bmat([[mass_rt0, -div.T], [div, None]], format="csc")

# get the degrees of freedom for each variable
dof_p, dof_q = div.shape

# assemble the right-hand side
rhs = np.zeros(dof_p + dof_q)
rhs[:dof_q] += bc_val + vector_source

We need to solve the linear system, PyGeoN provides a framework for that. The actual imposition of essential boundary conditions (flux boundary conditions) might change the symmetry of the global system, the class `pg.LinearSystem` preserves this structure by internally eliminating these degrees of freedom. Once the problem is solved, we extract the two solutions $q$ and $p$.

In [5]:
# solve the problem
ls = pg.LinearSystem(spp, rhs)
ls.flag_ess_bc(bc_ess, np.zeros(dof_q + dof_p))
x = ls.solve()

# extract the variables
q = x[:dof_q]
p = x[-dof_p:]

Since the computed $q$ is one value per facet of the grid, for visualization purposes we project the flux in each cell center as vector. We finally export the solution to be visualized by [ParaView](https://www.paraview.org/).

In [6]:
# post process variables
proj_q = rt0.eval_at_cell_centers(sd)
cell_q = (proj_q * q).reshape((3, -1), order="F")
cell_p = p0.eval_at_cell_centers(sd) * p

save = pp.Exporter(sd, "sol", folder_name="ex2")
save.write_vtu([("cell_p", cell_p), ("cell_q", cell_q)])